In [1]:
import numpy as np
import pandas as pd
import jieba as jb
import matplotlib.pyplot as plt
%matplotlib inline
import math
import time
import datetime
import os
import re
import io
import csv
import shutil
import sys
import mysql.connector as sql
from mysql.connector import MySQLConnection, Error
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
sys.setdefaultencoding('utf-8')

In [76]:
pdbc = sql.connect(host='localhost', database='taipeiall', user='root', password='apple')

"""

select EQIPnumber,direct,YMD,hour,avgspeed,avgPercent from WVZLKH60 where YMD between "2016-09-05" and "2016-09-11";
"""
table=pd.read_sql("show tables;",pdbc)
tablelist = table.copy()
tablelist=tablelist.ix[:,0].tolist()

In [77]:
tablelist=pd.DataFrame(tablelist)

In [78]:
"""tablelist
tablelist.columns=['tablename']
tablelist['vd'] = tablelist.ix[:,0].str[1:]
tablelist['direct'] = tablelist.ix[:,0].str[:1]
pattern= {'direct':{u'E':'東',u'W':'西',u'S':'南',u'N':'北',u'U':'未指定'}}
tablelist=tablelist.replace(pattern,regex=True);
tablelist.to_excel("DB.xlsx",index = False)"""

In [12]:
'''query = """select EQIPnumber,direct,YMD,hour,avgspeed,avgPercent \
from %s where YMD between "2016-09-05" and "2016-09-11";"""
frames = []
for i in tablelist:
    temp=pd.read_sql(query%i,pdbc)
    frames.append(temp)
dbre=pd.concat(frames,ignore_index=True)
dbresult = dbre.copy()
dbresult.EQIPnumber = dbresult.EQIPnumber.str.decode('utf-8')
dbresult.direct = dbresult.direct.str.decode('utf-8')
dbresult.YMD = dbresult.YMD.str.decode('utf-8')
dbresult.hour = dbresult.hour.str.decode('utf-8')
dbresult['YMD']=pd.to_datetime(dbresult['YMD']);
dbresult['weekday'] = dbresult['YMD'].dt.dayofweek;
hour_to_num={'00:00:00':0,'01:00:00':1,'02:00:00':2,'03:00:00':3,'04:00:00':4,'05:00:00':5,'06:00:00':6,'07:00:00':7,'08:00:00':8,'09:00:00':9,'10:00:00':10,'11:00:00':11,'12:00:00':12,'13:00:00':13,'14:00:00':14,'15:00:00':15,'16:00:00':16,'17:00:00':17,'18:00:00':18,'19:00:00':19,'20:00:00':20,'21:00:00':21,'22:00:00':22,'23:00:00':23}

pattern= {'direct':{u'.?東$':'E',u'.?西$':'W',u'.?南$':'S',u'.?北$':'N',u'.向放置':'U'},'hour':hour_to_num}
dbresult=dbresult.replace(pattern,regex=True);
dbresult=dbresult.drop('YMD',axis = 1)

dbresult=dbresult.rename(index=str, columns={"EQIPnumber": "vd", "hour": "time", \
                                             "avgspeed": "speed", "avgPercent": "accu"})'''

In [191]:
testcsv = pd.read_csv('cj20170227.csv')
testcsv=testcsv.drop(testcsv.columns[[0]], axis=1)

In [216]:
testcsv['speed']=pd.DataFrame({"speed":np.random.rand(len(testcsv.jampercent))*80})
testcsv['accu']=pd.DataFrame({"accu":np.random.rand(len(testcsv.jampercent))*10})

In [218]:
# testcsv

In [2]:
def read_vd_info_excel(xls):
    vd_info_excel_df=pd.read_excel(xls,encoding='utf-8');
    vd_info_excel_dfc=vd_info_excel_df.copy()

    pattern= {'direct':{u'.?東$':'東',u'.?西$':'西',u'.?南$':'南',u'.?北$':'北',u'.向放置':'未指定'}\
              ,'newdirect':{u'.?東.?':'東',u'.?西.?':'西',u'.?南.?':'南',u'.?北.?':'北',u'.向放置':'未指定'}}

    vd_info_excel_dfc=vd_info_excel_dfc.replace(pattern,regex=True);
    return vd_info_excel_dfc
    


def db_table_list(db_connection):
    
    # all table in trafficall Database
    db_table_df=pd.read_sql('show tables;', con=db_connection)
    db_table_dfc=db_table_df.copy()
    db_table_dfc.columns=['tablename']
    db_table_dfc['vd'] = db_table_dfc.ix[:,0].str[1:]
    db_table_dfc['direct'] = db_table_dfc.ix[:,0].str[:1]
    pattern= {'direct':{u'E':'東',u'W':'西',u'S':'南',u'N':'北',u'U':'未指定'}}
    db_table_dfc=db_table_dfc.replace(pattern,regex=True);
    return db_table_dfc

def merge_table(pdbc,tempalte):
    #把資料庫內的表的名稱全部拉出來，

    tablename=db_table_list(pdbc)
    print tablename.count()

    # 這個excel檔，是有新的路名跟新的方向，要用此檔案呈現給客戶端。
    vd_info_excel=read_vd_info_excel(tempalte)
    print vd_info_excel.count()

    #把這兩個檔案結合，按照VD編號跟舊方向，去做連接，產生的新表的表明再回去DB做QUERY。
    vd_match=pd.merge(vd_info_excel,tablename , on=['vd','direct'])
    vd_match.sortby = vd_match.index
    vd_matchc=vd_match.copy()
    print vd_matchc.count()
    vd_sortby_direct=vd_matchc.ix[:,['sortby','vd','direct']]
    sortbylist=vd_matchc.sortby.tolist()
    vd_table_list=vd_matchc.ix[:,'tablename'].tolist()
    return vd_matchc,vd_sortby_direct,sortbylist,vd_table_list
 
def query_table_out(table_list,vd_sortby_direct,db_connection):
    def decode_to_utf8(df):
        df.EQIPnumber=df.EQIPnumber.str.decode("utf-8")
        df.location=df.location.str.decode("utf-8")
        df.direct=df.direct.str.decode("utf-8")
        df.YMD=df.YMD.str.decode("utf-8")
        df.hour=df.hour.str.decode("utf-8")
        return df
    import time
    tStart = time.time()
    frames = []
    for ele in table_list:
        frames.append(pd.read_sql('SELECT * FROM {r}'.format(r=ele), con=db_connection))

    result = pd.concat(frames, ignore_index=True)

    df_all=decode_to_utf8(result) 
        
    df_all=df_all.drop(['scooterflow','truckflow','addTotal','carflow','avgCarSpace','laneNumber'],axis=1)
    hour_to_num={'00:00:00':0,'01:00:00':1,'02:00:00':2,'03:00:00':3,'04:00:00':4,'05:00:00':5,'06:00:00':6,'07:00:00':7,'08:00:00':8,'09:00:00':9,'10:00:00':10,'11:00:00':11,'12:00:00':12,'13:00:00':13,'14:00:00':14,'15:00:00':15,'16:00:00':16,'17:00:00':17,'18:00:00':18,'19:00:00':19,'20:00:00':20,'21:00:00':21,'22:00:00':22,'23:00:00':23}

    pattern= {'direct':{u'.?東$':'東',u'.?西$':'西',u'.?南$':'南',u'.?北$':'北',u'.向放置':'未指定'},'hour':hour_to_num}
    df_all=df_all.replace(pattern,regex=True);
    df_all['YMD']=pd.to_datetime(df_all['YMD']);
    df_all['weekday'] = df_all['YMD'].dt.dayofweek;
    df_all.columns = ['vd','location','direct','YMD','time','eq','speed','accu','weekday']
    vd_data_sortby=pd.merge(vd_sortby_direct, df_all, on=['vd','direct'])
    
    tEnd = time.time()
    print "It cost %f sec" % (tEnd - tStart)
    return vd_data_sortby


def jamp_tamplate(sortbylist,vd_match_merge,predict_template,vd_data_sortby):
    tempframe = []
    import time
    tStart = time.time()
    for i in sortbylist:
        print i
        template = predict_template.copy()
        wwp=vd_data_sortby.where(vd_data_sortby.sortby==i).dropna().copy()
        sp4=wwp.speed.quantile(0.4);sp4
        wwpcjam=wwp.where((wwp.speed<sp4)&(wwp.accu>5)).dropna().copy()
        template.ix[:,'sortby']=i;
        for dd,tt in template[['weekday','time']].values.tolist():
            alldaynum=wwp[(wwp.weekday==dd)&(wwp.time==tt)].count()[0]
            jamdaynum=wwpcjam[(wwpcjam.weekday==dd)&(wwpcjam.time==tt)].count()[0]
            if alldaynum==0:
                template.ix[(template.time==tt)&(template.weekday==dd),'jamp']=0
            else:
                template.ix[(template.time==tt)&(template.weekday==dd),'jamp']=round((float(jamdaynum)/alldaynum),4);
        tempframe.append(pd.merge(template,vd_match_merge,on=['sortby'],copy=False))
    
    nhzctest=pd.concat(tempframe, ignore_index=True);
    nhzctest=nhzctest.rename(index=str,columns= {'jamp':'jampercent','newdirect':'direction'})
    tEnd = time.time()
    print "It cost %f sec" % (tEnd - tStart)
    return nhzctest

In [88]:
pdbc = sql.connect(host='localhost', database='taipeiall', user='root', password='apple')
excel='vd_real_info.xlsx'
predict_template=pd.read_excel('predict_template.xlsx',encoding='utf-8');

vd_match_merge,vd_sortby_direct,sortbylist,vd_table_list = merge_table(pdbc,excel)

vd_data_sortby = query_table_out(vd_table_list,vd_sortby_direct,pdbc)

# vd_jamp_template=jamp_tamplate(sortbylist,vd_match_merge,predict_template,vd_data_sortby)



tablename    853
vd           853
direct       853
dtype: int64
vd           714
direct       714
newdirect    714
road         714
position     714
district     714
city         714
sortby         0
dtype: int64
vd           656
direct       656
newdirect    656
road         656
position     656
district     656
city         656
sortby       656
tablename    656
dtype: int64
It cost 657.905861 sec


In [91]:
vd_data_sortby

In [220]:
sortbylist=testcsv.sortby.unique().tolist()

In [221]:
# 雍塞度需要公式，要在與塞車率融合

template = predict_template.copy()
import time
tStart = time.time()
# for i in sortbylist:
#     print i
i=3
wwp=vd_data_sortby.where(vd_data_sortby.sortby==i).dropna().copy()
sp4=wwp.speed.quantile(0.4);sp4

speed_min = wwp.speed.min()
accu_max = wwp.accu.max()

accu_lenth = accu_max - 5
speed_lenth = sp4 - speed_min

for dd,tt in template[['weekday','time']].values.tolist():
    # 條件判斷 
    condition_sort =(testcsv.sortby==i)&(testcsv.time==tt)&(testcsv.weekday==dd)
#     print (dd,tt)
    if  len(testcsv.ix[condition_sort,'accu'].values) ==1:
        predict_accu = testcsv.ix[condition_sort,'accu'].values[0]
        predict_speed = testcsv.ix[condition_sort,'speed'].values[0]
        #算雍塞度
        xp = predict_accu - 5
        yp = sp4 - predict_speed 
        congestion = (xp/accu_lenth)/2+(yp/speed_lenth)/2
        if (predict_accu>5)&(predict_speed<sp4):
            testcsv.ix[condition_sort,'congest']=round(congestion,4)
        else:
            pass
        

tEnd = time.time()
print "It cost %f sec" % (tEnd - tStart)


It cost 6.950671 sec


In [223]:
condition_sort =(testcsv.sortby==3)&(testcsv.time==6)&(testcsv.weekday==3)
print len(testcsv.ix[condition_sort].values)

1


In [224]:
testcsv.ix[condition_sort]

,sortby,weekday,time,jampercent,vd,direction,road,position,district,city,tablename,congest,direct,speed,accu
582,3,3,6,0.0,V0150C1,東,市民大道高架,市高敦化西側往東,Daan,Teipai,UV0150C1,0.0,U,68.982391,0.671051


In [226]:
testcsv.ix[(testcsv.sortby==3)&(testcsv.weekday==3)]

,sortby,weekday,time,jampercent,vd,direction,road,position,district,city,tablename,congest,direct,speed,accu
576,3,3,0,0.0000,V0150C1,東,市民大道高架,市高敦化西側往東,Daan,Teipai,UV0150C1,0.0000,U,42.598503,2.888166
577,3,3,1,0.0000,V0150C1,東,市民大道高架,市高敦化西側往東,Daan,Teipai,UV0150C1,0.0000,U,59.530255,4.928587
578,3,3,2,0.0000,V0150C1,東,市民大道高架,市高敦化西側往東,Daan,Teipai,UV0150C1,0.0000,U,62.669957,2.276259
579,3,3,3,0.0000,V0150C1,東,市民大道高架,市高敦化西側往東,Daan,Teipai,UV0150C1,0.2386,U,44.860488,5.800620
580,3,3,4,0.0000,V0150C1,東,市民大道高架,市高敦化西側往東,Daan,Teipai,UV0150C1,0.3249,U,36.435328,5.062754
581,3,3,5,0.0000,V0150C1,東,市民大道高架,市高敦化西側往東,Daan,Teipai,UV0150C1,0.0000,U,24.669390,3.053031
582,3,3,6,0.0000,V0150C1,東,市民大道高架,市高敦化西側往東,Daan,Teipai,UV0150C1,0.0000,U,68.982391,0.671051
583,3,3,7,0.0000,V0150C1,東,市民大道高架,市高敦化西側往東,Daan,Teipai,UV0150C1,0.0000,U,63.597676,0.876011
584,3,3,8,0.1750,V0150C1,東,市民大道高架,市高敦化西側往東,Daan,Teipai,UV0150C1,0.1533,U,57.112968,9.625575
585,3,3,9,0.9250,V0150C1,東,市民大道高架,市高敦化西側往東,Daan,Teipai,UV0150C1,0.0000,U,41.751399,1.975980


In [ ]:
vd_jamp_template.to_csv('cj20170227.csv',index=False)


In [ ]:
# 只有星期跟時間，只是用來把塞車率的各個時間塞進去。



In [ ]:
vd_jamp_template